# Un modelo matemático (Uncapacitated Lot Sizing)

El modelo equivale a la siguiente formulación:

\begin{align}
\text{minimize} \qquad & \sum_{t\in T} c_s s_t+ \sum_{t\in T} c_K y_t \\
 \text{subject to} \quad \quad & s_{t-1} + x_t = s_t + d_t \\
 \qquad \qquad & x_t \leq M y_t \\
 \qquad \qquad & s_0 = 0 \\
 \qquad \qquad & x_t, s_t \in \mathbb{R}^+, \forall t\in T\\
 \qquad \qquad & y_t \in \{0,1\}
\end{align}

$c_s$ y $c_K$ son, respectivamente, los costes de inventario (unidad por periodo) y de realizar un pedido en un periodo y $d_t$ la demanda de cada periodo. El conjunto de periodos es $T$. $x_t$ es la cantidad que se recibe en el periodo $t$, mientras que $y_t$ equivale a si se realiza un pedido en el periodo $t$ o no y $s_t$ es el inventario al final del periodo $t$.

El objetivo minimiza la suma de costos de inventario y pedido (el costo por unidad no se tiene en cuenta porque la solución óptima siempre compra exactamente la demanda, $\sum_{t\in T} d_t$.

La primera restricción asegura el cumplimiento de la demanda de cada periodo. La segunda restricción asegura que sólo se compran unidades si se ha realizado un pedido ($M$ es un valor lo suficientemente grande, en este caso basta con $\sum_{t\in T} d_t$). La tercera restricción inicializa el inventario y el resto de restricciones definen el problema.

In [ ]:
using JuMP
using GLPKMathProgInterface

In [ ]:
T=10
d=[5; 10; 7; 11; 13; 14; 2; 8; 17; 21]
cK=50
cs=2
M=sum(d)

In [ ]:
modelo=Model(solver = solver = GLPKSolverMIP())
@variable(modelo, x[1:T]>= 0)
@variable(modelo, y[1:T],Bin)
@variable(modelo, s[0:T]>=0)

#función objetivo a minimizar
@objective(modelo, Min, sum(cs*s[t] for t in 1:T)+sum(cK*y[t] for t in 1:T))

for t in 1:T
    @constraint(modelo,s[t-1]+x[t]==d[t]+s[t])
end
for t in 1:T
    @constraint(modelo,x[t]<=M*y[t])
end
@constraint(modelo,s[0]==0)

#escribimos el modelo
print(modelo)

#resolvemos
status = solve(modelo)

#mostramos resultados
println("**** Status: ",status," Objective value: ", getobjectivevalue(modelo))
println("**** x = ", getvalue(x))
println("**** y = ", getvalue(y))
println("**** s = ", getvalue(s))